In [6]:
from google.colab import drive
drive.mount('/content/drive')
import kagglehub
import os
from sklearn.model_selection import train_test_split
import random
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from tqdm import tqdm

nikhilroxtomar_person_segmentation_path = kagglehub.dataset_download('nikhilroxtomar/person-segmentation')
tapakah68_supervisely_filtered_segmentation_person_dataset_path = kagglehub.dataset_download('tapakah68/supervisely-filtered-segmentation-person-dataset')

print('Data source import complete.')
print(nikhilroxtomar_person_segmentation_path)
print(tapakah68_supervisely_filtered_segmentation_person_dataset_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using Colab cache for faster access to the 'person-segmentation' dataset.
Using Colab cache for faster access to the 'supervisely-filtered-segmentation-person-dataset' dataset.
Data source import complete.
/kaggle/input/person-segmentation
/kaggle/input/supervisely-filtered-segmentation-person-dataset


In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:1]:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/person-segmentation/people_segmentation/README
/kaggle/input/person-segmentation/people_segmentation/segmentation/val.txt
/kaggle/input/person-segmentation/people_segmentation/images/pexels-photo-219004.jpg
/kaggle/input/person-segmentation/people_segmentation/masks/girl-beautiful-beautiful-girl-face.png
/kaggle/input/supervisely-filtered-segmentation-person-dataset/df.csv
/kaggle/input/supervisely-filtered-segmentation-person-dataset/supervisely_person_clean_2667_img/supervisely_person_clean_2667_img/collage/ds6_pexels-photo-792385.jpg
/kaggle/input/supervisely-filtered-segmentation-person-dataset/supervisely_person_clean_2667_img/supervisely_person_clean_2667_img/images/ds8_pexels-photo-838817.png
/kaggle/input/supervisely-filtered-segmentation-person-dataset/supervisely_person_clean_2667_img/supervisely_person_clean_2667_img/masks/ds8_pexels-photo-838817.png


In [8]:
images_path =  "/kaggle/input/supervisely-filtered-segmentation-person-dataset/supervisely_person_clean_2667_img/supervisely_person_clean_2667_img/"
masks_path  = "/kaggle/input/supervisely-filtered-segmentation-person-dataset/supervisely_person_clean_2667_img/supervisely_person_clean_2667_img/"

images_path2="/kaggle/input/person-segmentation/people_segmentation/images/"
masks_path2="/kaggle/input/person-segmentation/people_segmentation/masks/"
df =  pd.read_csv('/kaggle/input/supervisely-filtered-segmentation-person-dataset/df.csv')
df.head()
images_path2_list = sorted(os.listdir(images_path2))
masks_path2_list = sorted(os.listdir(masks_path2))
df2 = df[["images", "masks"]].copy()
df2["images"] = df2['images'].apply(lambda x: images_path + x)
df2["masks"]  = df2['masks'].apply(lambda x: masks_path + x)
df2["coef"]   = 1

df3 = pd.DataFrame({
    "images": [images_path2 + elt for elt in images_path2_list],
    "masks":  [masks_path2 + elt for elt in masks_path2_list],
    "coef":   255
})

final_df = pd.concat([df2, df3], ignore_index=True)

X_train_raw, X_test_raw  =  train_test_split(final_df, test_size=0.1, random_state=42)

In [9]:
final_df = final_df.copy()
final_df["source"] = np.where(final_df["coef"] == 1, "supervisely", "person_seg")
print("Tổng số mẫu:", len(final_df))
print("\nSố mẫu theo nguồn:")
print(final_df["source"].value_counts())

Tổng số mẫu: 8345

Số mẫu theo nguồn:
source
person_seg     5678
supervisely    2667
Name: count, dtype: int64


In [10]:
def dice_loss(logits, target, eps=1e-6):
    pred = torch.sigmoid(logits)
    target = target.float()

    pred = pred.view(pred.size(0), -1)
    target = target.view(target.size(0), -1)

    inter = (pred * target).sum(dim=1)
    denom = pred.sum(dim=1) + target.sum(dim=1)
    dice = (2 * inter + eps) / (denom + eps)
    return 1 - dice.mean()

def dice_score_from_logits(logits, target, thr=0.5, eps=1e-6):
    prob = torch.sigmoid(logits)
    pred = (prob > thr).float()
    inter = (pred * target).sum()
    union = pred.sum() + target.sum()
    return ((2*inter + eps) / (union + eps)).item()

def logits_to_probs_preds(logits, thr=0.5):
    probs = torch.sigmoid(logits)
    preds = (probs > thr).float()
    return probs, preds
def update_global_stats_from_logits(logits, targets, thr=0.5):
    probs = torch.sigmoid(logits)
    preds = (probs > thr).float()

    tp = (preds * targets).sum().item()
    fp = (preds * (1 - targets)).sum().item()
    fn = ((1 - preds) * targets).sum().item()

    correct = (preds == targets).float().sum().item()
    total = targets.numel()

    return tp, fp, fn, correct, total


In [ ]:
class RawSegDataset(Dataset):
    def __init__(self, df, size=512):
        self.df = df.reset_index(drop=True)
        self.size = size
        self.mean = torch.tensor([0.485, 0.456, 0.406], dtype=torch.float32).view(3,1,1)
        self.std  = torch.tensor([0.229, 0.224, 0.225], dtype=torch.float32).view(3,1,1)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.loc[idx]
        img_path  = row["images"]
        mask_path = row["masks"]

        img = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        img = img.resize((self.size, self.size), Image.BILINEAR)
        mask = mask.resize((self.size, self.size), Image.NEAREST)

        img_np = np.array(img, dtype=np.float32)
        mask_np = np.array(mask, dtype=np.float32)

        max_val = mask_np.max() if mask_np.max() > 0 else 1.0
        if max_val > 1.0:
            mask_np = mask_np / max_val
        mask_bin = (mask_np >= 0.5).astype(np.float32)

        img_np = img_np / 255.0

        img_t = torch.from_numpy(img_np).permute(2,0,1).float()
        mask_t = torch.from_numpy(mask_bin).unsqueeze(0).float()

        img_t = (img_t - self.mean) / self.std

        return img_t, mask_t

class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.net(x)

class Down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.net = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_ch, out_ch)
        )

    def forward(self, x):
        return self.net(x)

class Up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=True):
        super().__init__()
        self.bilinear = bilinear

        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=False)
            self.conv = DoubleConv(in_ch, out_ch)
        else:
            self.up = nn.ConvTranspose2d(in_ch // 2, in_ch // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_ch, out_ch)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        diffY = x2.size(2) - x1.size(2)
        diffX = x2.size(3) - x1.size(3)
        if diffY != 0 or diffX != 0:
            x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                            diffY // 2, diffY - diffY // 2])

        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, in_channels=3, num_classes=1, base_c=64, bilinear=True):
        super().__init__()
        self.in_conv = DoubleConv(in_channels, base_c)
        self.down1 = Down(base_c, base_c * 2)
        self.down2 = Down(base_c * 2, base_c * 4)
        self.down3 = Down(base_c * 4, base_c * 8)

        factor = 2 if bilinear else 1
        self.down4 = Down(base_c * 8, base_c * 16 // factor)

        self.up1 = Up(base_c * 16, base_c * 8 // factor, bilinear=bilinear)
        self.up2 = Up(base_c * 8, base_c * 4 // factor, bilinear=bilinear)
        self.up3 = Up(base_c * 4, base_c * 2 // factor, bilinear=bilinear)
        self.up4 = Up(base_c * 2, base_c, bilinear=bilinear)

        self.out_conv = nn.Conv2d(base_c, num_classes, kernel_size=1)

    def forward(self, x):
        x1 = self.in_conv(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)

        x = self.up1(x5, x4)
        x = self.up2(x,  x3)
        x = self.up3(x,  x2)
        x = self.up4(x,  x1)

        logits = self.out_conv(x)
        return logits

In [16]:
train_ds = RawSegDataset(X_train_raw, size=256)
val_ds   = RawSegDataset(X_test_raw,  size=256)

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=8, shuffle=False, num_workers=2, pin_memory=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet(in_channels=3, num_classes=1, base_c=32, bilinear=True).to(device)

criterion_bce = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 20
history = {
    "train_loss": [],
    "val_loss": [],
    "val_iou": [],
    "val_dice": [],
    "val_acc": [],
    "val_prec": [],
    "val_recall": [],
    "val_f1": []
}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    tp = fp = fn = 0.0
    correct = 0.0
    total = 0

    for imgs, masks in tqdm(train_loader, desc=f"Train epoch {epoch+1}"):
        imgs = imgs.to(device).float()
        masks = masks.to(device).float()

        logits = model(imgs)

        bce = criterion_bce(logits, masks)
        dsc = dice_loss(logits, masks)
        loss = bce + dsc

        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        with torch.no_grad():
            _tp, _fp, _fn, _correct, _total = update_global_stats_from_logits(logits, masks, thr=0.5)
            tp += _tp
            fp += _fp
            fn += _fn
            correct += _correct
            total += _total

    avg_loss = running_loss / max(1, len(train_loader))

    train_iou  = tp / (tp + fp + fn + 1e-6)
    train_dice = (2*tp) / (2*tp + fp + fn + 1e-6)
    train_acc  = correct / max(1, total)

    print(f"Epoch {epoch+1} - train avg loss: {avg_loss:.4f} | IoU: {train_iou:.4f} | Dice: {train_dice:.4f} | Acc: {train_acc:.4f}")
    history["train_loss"].append(avg_loss)

    model.eval()
    val_loss = 0.0
    val_steps = 0

    v_tp = v_fp = v_fn = 0.0
    v_correct = 0.0
    v_total = 0

    with torch.no_grad():
        for imgs, masks in val_loader:
            imgs = imgs.to(device).float()
            masks = masks.to(device).float()

            logits = model(imgs)

            bce = criterion_bce(logits, masks)
            dsc = dice_loss(logits, masks)
            batch_loss = (bce + dsc).item()
            val_loss += batch_loss
            val_steps += 1

            _tp, _fp, _fn, _correct, _total = update_global_stats_from_logits(logits, masks, thr=0.5)
            v_tp += _tp
            v_fp += _fp
            v_fn += _fn
            v_correct += _correct
            v_total += _total

    avg_val_loss = val_loss / max(1, val_steps)

    val_iou  = v_tp / (v_tp + v_fp + v_fn + 1e-6)
    val_dice = (2*v_tp) / (2*v_tp + v_fp + v_fn + 1e-6)
    val_acc  = v_correct / max(1, v_total)

    val_prec   = (v_tp + 1e-6) / (v_tp + v_fp + 1e-6)
    val_recall = (v_tp + 1e-6) / (v_tp + v_fn + 1e-6)
    val_f1     = (2*val_prec*val_recall + 1e-6) / (val_prec + val_recall + 1e-6)

    history["val_loss"].append(avg_val_loss)
    history["val_iou"].append(val_iou)
    history["val_dice"].append(val_dice)
    history["val_acc"].append(val_acc)
    history["val_prec"].append(val_prec)
    history["val_recall"].append(val_recall)
    history["val_f1"].append(val_f1)

    print(f"Val loss: {avg_val_loss:.4f} | IoU: {val_iou:.4f} | Dice: {val_dice:.4f} | Acc: {val_acc:.4f}")
    print(f"Precision: {val_prec:.4f} | Recall: {val_recall:.4f} | F1: {val_f1:.4f}")


hist_df = pd.DataFrame(history)
hist_df.to_csv("/content/drive/MyDrive/Data Mining/Project/Model/training_unet_history_raw_metrics.csv", index=False)
print("Saved training history to Drive (training_unet_history_raw_metrics.csv)")
save_path = "/content/drive/MyDrive/Data Mining/Project/Model/unet_raw.pth"
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}, save_path)
print(f"Model saved to {save_path}")


Train epoch 1: 100%|██████████| 939/939 [03:55<00:00,  3.99it/s]

Epoch 1 - train avg loss: 0.8513 | IoU: 0.5713 | Dice: 0.7271 | Acc: 0.8410


Val loss: 0.6778 | IoU: 0.6516 | Dice: 0.7890 | Acc: 0.8863
Precision: 0.7966 | Recall: 0.7816 | F1: 0.7890


Train epoch 2: 100%|██████████| 939/939 [03:54<00:00,  4.00it/s]

Epoch 2 - train avg loss: 0.6131 | IoU: 0.6694 | Dice: 0.8019 | Acc: 0.8890


...

Epoch 19 - train avg loss: 0.1122 | IoU: 0.9364 | Dice: 0.9672 | Acc: 0.9819


Val loss: 0.2460 | IoU: 0.8671 | Dice: 0.9288 | Acc: 0.9613
Precision: 0.9296 | Recall: 0.9280 | F1: 0.9288


Train epoch 20: 100%|██████████| 939/939 [03:53<00:00,  4.03it/s]

Epoch 20 - train avg loss: 0.1067 | IoU: 0.9391 | Dice: 0.9686 | Acc: 0.9827


Val loss: 0.2632 | IoU: 0.8600 | Dice: 0.9247 | Acc: 0.9588
Precision: 0.9188 | Recall: 0.9308 | F1: 0.9247
Saved training history to Drive (training_history_raw_metrics.csv)
Model saved to /content/drive/MyDrive/Data Mining/Project/Model/unet_raw.pth
